# Importa librerías

In [1]:
# Import libraries
import os
import requests
import pandas as pd
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.orchestration.metadata import sqlite_metadata_connection_config
from tensorflow_metadata.proto.v0 import statistics_pb2, schema_pb2
import tensorflow_data_validation as tfdv
from sklearn.model_selection import train_test_split
print('TF version:', tf.__version__)
print('TFDV version:', tfdv.version.__version__)

2025-02-26 02:15:46.654845: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 02:15:46.655855: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 02:15:46.658871: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-26 02:15:46.665618: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-26 02:15:46.677288: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registe

TF version: 2.16.2
TFDV version: 1.16.1


In [2]:
from data_preparation import *
from model_creation import *
from sklearn.feature_selection import SelectKBest, chi2, f_classif

# 2.1 Lectura de datos

In [3]:
## download the dataset
# Directory of the raw data files
_data_root = './data/covertype'
# Path to the raw training data
_data_filepath = os.path.join(_data_root, 'covertype_train.csv')

# Download data
os.makedirs(_data_root, exist_ok=True)
if not os.path.isfile(_data_filepath):
    #https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/
    url = 'https://docs.google.com/uc?export= \
    download&confirm={{VALUE}}&id=1lVF1BCWLH4eXXV_YOJzjR7xZjj-wAGj9'
    r = requests.get(url, allow_redirects=True, stream=True)
    open(_data_filepath, 'wb').write(r.content)
    
df = cargar_datos(_data_filepath)

Elevation                              int64
Aspect                                 int64
Slope                                  int64
Horizontal_Distance_To_Hydrology       int64
Vertical_Distance_To_Hydrology         int64
Horizontal_Distance_To_Roadways        int64
Hillshade_9am                          int64
Hillshade_Noon                         int64
Hillshade_3pm                          int64
Horizontal_Distance_To_Fire_Points     int64
Wilderness_Area                       object
Soil_Type                             object
Cover_Type                             int64
dtype: object


In [4]:
# Se seleccionan solo las variables numéricas
df_numerico = df.select_dtypes(include=['number'])

In [5]:
df_numerico.dtypes

Elevation                             int64
Aspect                                int64
Slope                                 int64
Horizontal_Distance_To_Hydrology      int64
Vertical_Distance_To_Hydrology        int64
Horizontal_Distance_To_Roadways       int64
Hillshade_9am                         int64
Hillshade_Noon                        int64
Hillshade_3pm                         int64
Horizontal_Distance_To_Fire_Points    int64
Cover_Type                            int64
dtype: object

In [6]:
X = df_numerico.drop(['Cover_Type'], axis=1)
y = df_numerico['Cover_Type']

# 3. Selección de características

In [7]:
# Aplicar SelectKBest
selector = SelectKBest(f_classif, k=8)
X_new = selector.fit_transform(X, y)

# Obtener los nombres de las columnas seleccionadas
columnas_seleccionadas = X.columns[selector.get_support()]

# Convertir nuevamente en DataFrame
X_new = pd.DataFrame(X_new, columns=columnas_seleccionadas, index=X.index)

In [8]:
# Suponiendo que X_train es un DataFrame y y_train es una Serie o DataFrame
df_new = pd.concat([X_new, y], axis=1)

In [9]:
# Se guardan los datos en ruta definida
root_path = './data/data_new'
file_path = os.path.join(root_path,"cover_new.csv")
df_new.to_csv(file_path)

# 4. Data Pipeline

## 4.1. Configurar el contexto interactivo


In [10]:
# Define un directorio persistente para los artefactos del pipeline
pipeline_root = "/ruta/persistente/para/pipeline_outputs"

# Inicia el contexto interactivo con la ruta definida
context = InteractiveContext(pipeline_root=pipeline_root)

# Define una ruta para la base de datos SQLite
metadata_path = "/ruta/persistente/para/metadata.sqlite"

# Configura la conexión a la metadata
metadata_config = sqlite_metadata_connection_config(metadata_path)

# Inicia el contexto con metadata persistente
context = InteractiveContext(pipeline_root=pipeline_root, metadata_connection_config=metadata_config)


## 4.2. Generando Ejemplos

In [11]:
# Crear el componente ExampleGen
example_gen = CsvExampleGen(input_base=root_path)
context.run(example_gen)



ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 21
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## 4.3. Estadísticas


In [12]:
# Crear el componente StatisticsGen
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
context.run(statistics_gen)


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 22
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

In [13]:
stats_path = os.path.join(statistics_gen.outputs['statistics'].get()[0].uri,'Split-train','FeatureStats.pb')

# Leer el archivo en formato protobuf
with open(stats_path, "rb") as f:
    stats_proto = statistics_pb2.DatasetFeatureStatisticsList()
    stats_proto.ParseFromString(f.read())

# Visualizar estadísticas
tfdv.visualize_statistics(stats_proto)

## 4.4. Inferir el esquema


In [14]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'])
context.run(schema_gen)


ExecutionResult(
    component_id: SchemaGen
    execution_id: 23
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None, _is_async=False))

## 4.5. Curando el esquema


In [16]:
# Obtener la URI del esquema generado por SchemaGen
schema_uri = schema_gen.outputs['schema'].get()[0].uri

# Cargar el esquema en formato protobuf
schema = tfdv.load_schema_text(schema_uri + "/schema.pbtxt")
# Definir rangos para variables numéricas
tfdv.set_domain(schema, "Hillshade_9am", schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, "Hillshade_Noon", schema_pb2.IntDomain(min=0, max=255))
tfdv.set_domain(schema, "Slope", schema_pb2.IntDomain(min=0, max=90))

# Definir Cover_Type como categórica
cover_type_domain = schema_pb2.Schema().feature.add()
cover_type_domain.name = "Cover_Type"
cover_type_domain.type = schema_pb2.INT  # Mantiene el tipo INT
cover_type_domain.int_domain.CopyFrom(schema_pb2.IntDomain(min=0, max=6))  # Rango permitido
cover_type_domain.annotation.tag.append("categorical")  # Declarar como categórico

## 4.6. Entornos de esquema


In [18]:
# Seleccionar un subconjunto aleatorio (por ejemplo, el 20% de los datos)
df_service = df_new.sample(frac=0.2, random_state=42).copy()

# Eliminar la columna de la etiqueta (Cover_Type)
df_service.drop(columns=["Cover_Type"], inplace=True)

service_path = "./data/data_service"
file_service_path = os.path.join(service_path,'service_data.csv')
# Guardar el conjunto de datos de servicio
df_service.to_csv(file_service_path)

In [19]:
# Se valida que sin entorno se marca la columna Cover_Type
options = tfdv.StatsOptions(schema=schema, infer_type_from_schema=True)
serving_stats = tfdv.generate_statistics_from_csv(file_service_path, stats_options=options)
serving_anomalies = tfdv.validate_statistics(serving_stats, schema)

tfdv.display_anomalies(serving_anomalies)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


,Anomaly short description,Anomaly long description
Feature name,,
'Cover_Type',Column dropped,Column is completely missing


In [20]:
# Se utilizan los entornos
# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('SERVING')

# Specify that 'Cover_Type' feature is not in SERVING environment.
tfdv.get_feature(schema, 'Cover_Type').not_in_environment.append('SERVING')
serving_anomalies_with_env = tfdv.validate_statistics(serving_stats, schema, environment='SERVING')

tfdv.display_anomalies(serving_anomalies_with_env)

In [21]:
# Guardo el esquema
schema_file = 'schema.pb'
schema_folder = './schema'
schema_path = os.path.join(schema_folder,schema_file)
tfdv.write_schema_text(schema, schema_path)

In [22]:
# Muestro el esquema y los cambios que se introducieron
schema = tfdv.load_schema_text(schema_path)  # Para el archivo de texto

In [23]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'',INT,required,,-
'Cover_Type',INT,required,,-
'Elevation',INT,required,,-
'Hillshade_9am',INT,required,,min: 0; max: 255
'Hillshade_Noon',INT,required,,min: 0; max: 255
'Horizontal_Distance_To_Fire_Points',INT,required,,-
'Horizontal_Distance_To_Hydrology',INT,required,,-
'Horizontal_Distance_To_Roadways',INT,required,,-
'Slope',INT,required,,min: 0; max: 90


In [24]:
schema.default_environment

['TRAINING', 'SERVING']